<a target="_blank" href="https://colab.research.google.com/github/petzschnerlab/avoid_learning_rl_models/blob/main/docs/Tutorials/RL_tutorial.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# RL Modelling Tutorial

Welcome to the avoidance learning reinforcement learning models repo. This repo was built for the PEAC lab to computationally model empirical data from the various avoidance learning tasks. This repo has the ability to fit RL models to empirical data and to validate these models through parameters and model recovery methods.  

*Note that this tutorial is designed to run in Google Colab and not from the repo itself (since it clones the repo)*

<b>Supported models</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;QLearning, ActorCritic<br>
&nbsp;&nbsp;&nbsp;&nbsp;Relative, Advantage<br>
&nbsp;&nbsp;&nbsp;&nbsp;Hybrid2012, Hybrid2021<br>
&nbsp;&nbsp;&nbsp;&nbsp;StandardHybrid2012, StandardHybrid2021<br><br>

<b>Standard models </b><br>
Standard models as described in each reference, which introduces the model, with the addition of counterfactual learning rates.<br>
Hybrid models have alternatives versions without counterfactual learning rates: StandardHybrid2012, StandardHybrid2021
&nbsp;&nbsp;&nbsp;&nbsp;<b>QLearning</b>: Standard Q-Learning Model<br>
&nbsp;&nbsp;&nbsp;&nbsp;<b>ActorCritic</b>: Standard Actor-Critic Model<br>
&nbsp;&nbsp;&nbsp;&nbsp;<b>Relative</b>: Standard Relative Model (Palminteri et al., 2015)<br>
&nbsp;&nbsp;&nbsp;&nbsp;<b>Advantage</b>: Simplified Relative Model (Williams et al., 2025)<br>
&nbsp;&nbsp;&nbsp;&nbsp;<b>Hybrid2012+bias</b>: Hybrid 2012 Model (Gold et al., 2012)<br>
&nbsp;&nbsp;&nbsp;&nbsp;<b>Hybrid2021+bias+decay</b>: Hybrid 2021 Model (Geana et al., 2021)<br><br>

<b>Optional Parameters</b><br>
You can add optional parameters to models by adding them to the model name using a + sign<br>
&nbsp;&nbsp;&nbsp;&nbsp;<b>+bias</b>: Adds a valence bias to the model (e.g. Hybrid2012+bias), only usable with Hybrid models<br>
&nbsp;&nbsp;&nbsp;&nbsp;<b>+novel</b>: Adds a free parameter for the novel stimulus (e.g. QLearning+novel), useable with all models<br>
&nbsp;&nbsp;&nbsp;&nbsp;<b>+decay</b>: Adds a decay parameter to the model (e.g. QLearning+decay), useable with all models

## Project Pipeline
This repo is one part of a project pipeline, which requires the coordination of multiple repos. Projects begin with a <b>task repo</b>, which is used to collect behavioural data from participants either locally or on Prolific. The collected data must then be pushed through a <b>data extraction repo</b> to prepare CSV files for analysis. These CSV files are used in <b>the analysis repo</b>, which creates a PDF report (`AL/reports`), ending the project pipeline. 

Optionally, you can run computational reinforcement learning models using the <b>modelling repo (this repo)</b>, and the results can be added to the report. This is a bit clunky because it requires a bit of back-and-forth between the analysis repo and this modelling repo. Specifically, the analysis repo must be run (with `load_models=False`) in order to create two CSV files that this repo needs (`AL/data/pain_learning_processed.csv` and `AL/data/pain_transfer_processed.csv`). These files can then be manually moved into this repo's data directory (`RL/data`). This repo can then be used to model the data, which will result in a newly constructed directory called `modelling` (`RL/modelling`). This folder can then be manually moved to the analysis repo as `AL/modelling`. Then you can re-run the analysis repo (with `load_models=True`) and the modelling results will be included in the PDF report. 

## Cloning the Repo

We will begin by cloning the repo, installing dependencies, and then adding this repo as a system path. Adding the repo in the system path is only necessary for this tutorial. We also change directory to the repo. When using locally, you can create your script in the `AL` source folder, in the same manner as `AL_main.py` (`avoid_learning_analysis/AL/AL_main.py`).

In [ ]:
%%capture

import sys
import os

# We will now clone the repo, pull any updates, and install dependencies
!git clone https://github.com/petzschnerlab/avoid_learning_rl_models.git
%cd avoid_learning_rl_models/
!git pull
!pip install .

#Only necessary for Google Colab
sys.path.insert(0, os.path.abspath("/content/avoid_learning_rl_models/RL"))

## The Pipeline

Next, we will import the Pipeline class. This class is the entry point to this repo. It will take in all of your parameters and run the corresponding analyses.

In [ ]:
from helpers.pipeline import Pipeline

### The Help Function

The pipeline has a help function that will outline some information about the repo and then describe all of the parameters. These details are also available in the documentation. We will use the `help=True` parameters in order to see this help function below. 

This parameter can be passed to the Pipeline during initiatialization:
```
pipeline = Pipeline(help=True)
```

or to the pipeline run method of the class:
```
pipeline = Pipeline()
pipeline.run(help=True)
```

The help information gets truncated in Jupyter notebooks, but you can view the whole output by clicking `scrollable element`.

In [ ]:
pipeline = Pipeline(help=True)

## Fitting Models

There are two modes in this repo, `fit` and `validation`. In FIT mode, models are fitted to empirical data. In VALIDATION mode, parameter recovery or model recovery is performed, depending on the recovery parameter. We will begin by fitting several RL models to our empirical data. 

We will define a typical set of parameters for this package below, see the help information above to understand what each parameters does. Note that in this example, we extract our priors using the fixed_priors function. Note that here we will be using learning and transfer filenames `tutorial_learning_data.csv` and `tutorial_transfer_data.csv`, respectively, but the default filenames exported by the analysis repo are `pain_learning_processed.csv` and `pain_transfer_processed.csv`. Finally, it is highly recommended to use multiprocessing if possible as we are doing here (e.g., `multiprocessing=True`). Especially when you have a lot of participants, are running many models, and including many runs, this will take a very long time if you are not using multiprocessing. It will still take a good amount of time when using multiprocessing.

A final note is that we are setting a seed when initializing the pipeline to ensure replicability because this repo does use randomization.

In [ ]:
from helpers.priors import fixed_priors

models = ['QLearning+novel',    #Standard + novel
          'ActorCritic+novel',  #Standard + novel
          'Advantage+novel',    #Standard + novel
]
    
fixed = fixed_priors(models)    
fit_params = {'mode':                       'fit',
              'learning_filename':          'RL/data/tutorial_learning_data.csv',
              'transfer_filename':          'RL/data/tutorial_transfer_data.csv',
              'models':                     models,
              'random_params':              'normal',
              'fixed':                      fixed,
              'multiprocessing':            True,
}

pipeline = Pipeline(seed=1251)
pipeline.run(**fit_params)

### Fitting Results

Now that we have finished fitting the models to our data, we can look at a couple of plots to see the results. After the fitting procedure, a new folder is created `RL/modelling`. This folder contains all important results (statistics, plots, etc.). The intent of this folder is to move it to the analysis repo and that way you can add your modelling results to your PDF report. For this tutorial we will look at a couple of plots to see our results.

Let's begin by viewing the BIC plot, which will tell us which model fit the data the best. Remember, the lower this value the better the data fit. 

In [ ]:
from IPython.display import Image, display

display(Image(filename='RL/modelling/BIC-model-comparisons.png'))

We can also view the fits from the BIC table, which will also show us the best fits for each group.

In [ ]:
import pandas as pd

BIC_fits = pd.read_csv('RL/modelling/group_BIC.csv')
BIC_fits

The fitting procedure also runs model simulations using the fitted parameters of each participant. We can also view these simulation results. We will view the best models simulations.

In [ ]:
best_model = BIC_fits['best_model'].values[-1]
print(best_model)

display(Image(filename=f'RL/modelling/model_behaviours/{best_model}_model_behaviours.png', width=800, height=600))